Ноутбук должен быть создан в среде tensorflow.
Если её еще нет, то настроить среду можно в Anaconda Navigator следующим способом:
-зайти в "Environment",
-нажать "Create",
-дать имя среде, например, "tensorflow",
-нажать "Apply",
-справа выбрать опцию "Not installed", подождать несколько минут, пока подгрузятся директории с названиями, содержащими "tensorflow",
-выбрать "tensorflow"
-перейти в "Home",
-в блоке "Jupyter" нажать на "Install" и установить Jupyter notebook.

Для работы камеры требуется запускать ноутбук локально, у себя на компьютере

In [1]:
import os

In [2]:
!pip install pandas
import pandas as pd

In [7]:
!pip install opencv-python

In [10]:
!pip install matplotlib

In [3]:
import tensorflow as tf

In [4]:
from tensorflow.keras.models import load_model

In [5]:
pip install opencv-python
import cv2
from copy import deepcopy

In [6]:
import numpy as np
import matplotlib.pyplot as plt

In [32]:
import wget
wget.download('https://raw.githubusercontent.com/opencv/opencv/master/data/haarcascades/haarcascade_frontalface_default.xml')

In [79]:
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.applications.mobilenet_v2 import preprocess_input

In [199]:
class PREDICT_EMOTION(tf.keras.Model):
    def __init__(input_path):
        
        """
        input_path -- путь для загрузки весов модели
        """

        model = load_model(input_path)
        return model
    
    def predict_emotion(model, saving_output_path, loading_path, image_shape):
        cam = cv2.VideoCapture(0)
        if not cam.isOpened():
            print("Не удалось открыть камеру")
        else:
            print("Камера запущена")
        
        ret, frame = cam.read()
        if ret == True:
            plt.imshow(frame)
        bgr_image = frame
        bgr_image = cv2.resize(bgr_image, (image_shape, image_shape))
        rgb_image = cv2.cvtColor(bgr_image, cv2.COLOR_BGR2RGB)
#saving_output_path и loading_path важно разделить, так как flow_from_directory (ниже) будет считывать фотографии
#из папки более высокого уровня, пример:
#saving_output_path="saved_images/images", loading_path="saved_images", где loading_path является родительской директорией для  saving_output_path       
        face_detector = cv2.CascadeClassifier('haarcascade_frontalface_default.xml')
        faces = face_detector.detectMultiScale(grayscale_image, 1.3, 5)
        print("Найдено {} лицо/лиц".format(len(faces)))
        print("Массив faces: {}".format(faces))
        
        one_face = faces[0]
        x, y, w, h = one_face
        face_boundingbox_bgr = bgr_image[y:y + h, x:x + w]
        face_boundingbox_rgb = cv2.cvtColor(face_boundingbox_bgr, cv2.COLOR_BGR2RGB)

        
        path = saving_output_path
        cv2.imwrite(os.path.join(saving_output_path, 'loaded_image.jpg'), rgb_image)
        cv2.waitKey(0)
        
    #def read_frame():
        #while(True):
            #ret, frame = cam.read()
            #cv2.imshow("facial emotion recognition", frame)
            
            #if cv2.waitKey(1) & 0xFF == ord('q'):
                #break
                
        image_gen = ImageDataGenerator(preprocessing_function=preprocess_input)
        
        loaded_image_generator_emotion = image_gen.flow_from_directory(
            directory = loading_path,
            target_size=(image_shape, image_shape))
        
        predicted_classes = model.predict_classes(loaded_image_generator_emotion)
        classes_mapping = {0: 'anger', 1: 'contempt', 2: 'disgust', 3: 'fear', 4: 'happy', 5: 'neutral', 6: 'sad', 7: 'surprise', 8: 'uncertain'}
        
        path = saving_output_path
        for i in predicted_classes:
            emotion_text = str(classes_mapping[i])
   
            rgb_image = cv2.imread(os.path.join(saving_output_path, 'loaded_image.jpg'))
            rgb_image_with_boundingbox = deepcopy(rgb_image)
            rgb_image_with_boundingbox = cv2.rectangle(rgb_image_with_boundingbox, (y, x), (y + h, x + w), (0,255,0), 3)
            rgb_image_with_boundingbox_and_text = deepcopy(rgb_image_with_boundingbox)
            rgb_image_with_boundingbox_and_text = cv2.putText(rgb_image_with_boundingbox_and_text, emotion_text, (y, x - 10), cv2.FONT_HERSHEY_SIMPLEX, 0.5, (0, 255, 0), 2)
            plt.figure(figsize=(15, 5))
            plt.imshow(rgb_image_with_boundingbox_and_text)
            plt.title("Predicted emotion:")
            plt.show()

In [88]:
IMAGE_SHAPE = 224

In [200]:
model_emotion = PREDICT_EMOTION.__init__("Диплом/Submission_2_29.12.20/model_emotion/checkpoint_best.h5")

In [ ]:
PREDICT_EMOTION.predict_emotion(model_emotion, saving_output_path="saved_images/images", loading_path="saved_images", image_shape=IMAGE_SHAPE)